### Few words about Bamberg

In [1]:
import happygisco
from happygisco import settings, base, services, features

! missing FASTENERS package (https://pypi.org/project/fasteners/) <class 'ImportWarning'> !
! missing SIMPLEJSON package (https://pypi.python.org/pypi/simplejson/) <class 'ImportWarning'> !
! missing GOOGLEPLACES package (https://github.com/slimkrazy/python-google-places) !


Let's define the place:

In [2]:
place = 'Bamberg, Bavaria, Germany'

Run the geocoding service:

In [3]:
serv = services.GISCOService()
url=serv.url_geocode(q=place)
print(url)
geom = serv.place2geom(place)
print(geom)
coord = serv.place2coord(place)
print('The lat/Lon coordinates of %s are: %s' % (place,coord))

https://europa.eu/webtools/rest/gisco/api?q=Bamberg%2C+Bavaria%2C+Germany
[{'geometry': {'coordinates': [10.913098076467719, 49.920426199999994], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 300909509, 'osm_type': 'W', 'extent': [10.9057423, 49.9260091, 10.9241217, 49.9148577], 'country': 'Germany', 'osm_key': 'aeroway', 'city': 'Bamberg', 'street': 'Kemmerstraße', 'osm_value': 'aerodrome', 'postcode': '96052', 'name': 'Bamberg-Breitenau Airfield', 'state': 'Bavaria'}}, {'geometry': {'coordinates': [10.912427370672816, 49.901440449999996], 'type': 'Point'}, 'type': 'Feature', 'properties': {'osm_id': 31739467, 'osm_type': 'W', 'extent': [10.9120258, 49.9016719, 10.9128331, 49.9012357], 'country': 'Germany', 'osm_key': 'office', 'housenumber': '75', 'city': 'Bamberg', 'street': 'Pödeldorfer Straße', 'osm_value': 'insurance', 'postcode': '96052', 'name': 'AOK - Health insurance (Bamberg)', 'state': 'Bavaria'}}]
The lat/Lon coordinates of Bamberg, Bavaria, Germany are: [[

Let's identify the NUTS Bamberg belongs to:

In [4]:
nuts = serv.place2nuts(place)
units = [n['attributes']['NUTS_ID'] for n in nuts]
print('%s is located in NUTS %s' % (place,units))

Bamberg, Bavaria, Germany is located in NUTS ['DE2', 'DE241', 'DE24', 'DE']


A dummy map...:

In [5]:
NUTS = features.NUTS(unit=units)
NUTS.carto()

Let's retrieve some data from Eurostat:

In [6]:
import json, collections
import requests
import pandas
url = 'http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/lfst_r_lfu3rt?sex=T&precision=1&unit=PC&filterNonGeo=1&time=2017&shortLabel=1&age=Y_GE15&unitLabel=label'
resp = requests.session().get(url)
data = resp.json(object_pairs_hook=collections.OrderedDict)
idx2code = {v:k for k,v in data['dimension']['geo']['category']['index'].items()}
df = pandas.DataFrame([(idx2code[int(k)],v) for k,v in data['value'].items()], columns=['geo','value']).dropna()
df = df[df['geo'].str.startswith('DE')]
df.head()

,geo,value
59,DE,3.7
60,DE1,2.9
61,DE11,3.0
62,DE12,3.3
63,DE13,2.9


More data about NUTS at level 2:

In [7]:
import geopandas
NUTS2 = features.NUTS(unit=serv.nuts_info(info='UNITS', unit='DE', level=2))
gdf = NUTS2.load(ofmt='gpd')
gdf.head()

,CNTR_CODE,FID,LEVL_CODE,NUTS_ID,NUTS_NAME,geometry
0,DE,DE21K,3,DE21K,"Rosenheim, Landkreis","POLYGON ((12.388 48.117, 12.439 47.915, 12.338..."
1,DE,DE94H,3,DE94H,Wittmund,"POLYGON ((7.872 53.376, 7.815 53.375, 7.492 53..."
2,DE,DE40G,3,DE40G,Spree-Neiße,"POLYGON ((14.717 52.001, 14.73 51.582, 14.448 ..."
3,DE,DEA12,3,DEA12,"Duisburg, Kreisfreie Stadt","POLYGON ((6.81 51.455, 6.83 51.352, 6.805 51.3..."
4,DE,DEB12,3,DEB12,Ahrweiler,"POLYGON ((7.344 50.477, 6.984 50.349, 6.801 50..."


And here is a map of unemployment over Germany (for those data which are available at NUTS level 2):

In [8]:
import folium
map_display = folium.Map(location=settings.EU_GEOCENTRE, zoom_start=4)
map_display.choropleth(
    geo_data = gdf.to_json(), data=df,
    columns = ['geo', 'value'], key_on = 'feature.properties.NUTS_ID',
    name = 'choropleth',
    fill_color = 'YlOrRd', fill_opacity = 0.7, line_opacity = 0.2,
    legend_name = '2017 Unemployment Rate (%)'
)
map_display